<a href="https://colab.research.google.com/github/palxx/_100_days_of_CUDA/blob/main/cuda_day_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%writefile vector_multiplication.cu

Writing vector_multiplication.cu


In [2]:
%%writefile vector_multiplication.cu
#include <cuda_runtime.h>
#include <iostream>

__device__ float square(int x){
    return x*x;
    // __device__ marked function can only be called from anoter device function
    // or a kernel method
}

__global__ void voidKernel(int *input,int *output,int N) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < N){
        output[i] = square(input[i]);
    }
}
int main() {
    int N = 10; // Number of elements
    int size = N * sizeof(int);

    int *h_input = new int[N];
    int *h_output = new int[N];

    for (int i = 0; i < N; i++) {
        h_input[i] = i + 1;
    }

    int *d_input, *d_output;
    cudaMalloc((void**)&d_input, size);
    cudaMalloc((void**)&d_output, size);

    cudaMemcpy(d_input, h_input, size, cudaMemcpyHostToDevice);

    int threadsPerBlock = 256;
    int blockGrid = (N + threadsPerBlock - 1) / threadsPerBlock;

    // Start CUDA timing
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cudaEventRecord(start);

    // Run the kernel
    voidKernel<<<blockGrid, threadsPerBlock>>>(d_input, d_output, N);

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    // Measure time
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);

    // Copy results back to CPU
    cudaMemcpy(h_output, d_output, size, cudaMemcpyDeviceToHost);

    std::cout << "Execution Time: " << milliseconds << " ms" << std::endl;

    delete[] h_input;
    delete[] h_output;
    cudaFree(d_input);
    cudaFree(d_output);

    return 0;
}

Overwriting vector_multiplication.cu


In [3]:
!nvcc vector_multiplication.cu -o vector_multiplication

In [4]:
!./vector_multiplication

Execution Time: 100.574 ms


In [5]:
import torch
import triton
import triton.language as tl

@triton.jit
def __kernelfunction__(input_pointer, output_pointer, N, BLOCKSIZE: tl.constexpr):
    pid = tl.program_id(0)
    offset = pid * BLOCKSIZE + tl.arange(0, BLOCKSIZE)
    mask = offset < N
    input_data = tl.load(input_pointer + offset, mask=mask)
    output_data = input_data * input_data  # Squaring the input
    tl.store(output_pointer + offset, output_data, mask=mask)

def main():
    N = 10**6  # Larger array size for better benchmarking

    input_data = torch.arange(0, N, dtype=torch.float32, device="cuda")
    output_data = torch.empty_like(input_data, device="cuda")

    BLOCKSIZE = 256
    GRID = (triton.cdiv(N, BLOCKSIZE),)

    # Start timing
    start_event = torch.cuda.Event(enable_timing=True)
    stop_event = torch.cuda.Event(enable_timing=True)

    start_event.record()

    __kernelfunction__[GRID](input_data, output_data, N, BLOCKSIZE=BLOCKSIZE)

    stop_event.record()
    torch.cuda.synchronize()  # Ensure kernel execution finishes

    # Measure time
    elapsed_time = start_event.elapsed_time(stop_event)
    print(f"Execution Time: {elapsed_time:.4f} ms")

if __name__ == "__main__":
    main()

Execution Time: 3319.5745 ms
